In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

os.environ['JAVA_HOME'] = "/Users/ibrahimhuseynov/Library/Java/JavaVirtualMachines/ms-21.0.7/Contents/Home"

In [2]:
sys.executable

'/opt/anaconda3/envs/PythonProject3/bin/python'

In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName("Big Data Course at ADA") \
    .setMaster("local")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/28 21:52:03 WARN Utils: Your hostname, Ibrahims-MacBook-Air-3.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.111 instead (on interface en0)
25/11/28 21:52:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/28 21:52:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:

import os
import sys
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .appName("SEC_I_Dataset") \
    .getOrCreate()


df = spark.read \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .option("encoding", "UTF-8") \
    .csv("/Users/ibrahimhuseynov/Big_Data_1/Data_Collection/Merged_Csv/merged_1ci_qrup_corrected.csv")  

print("SEC I group SCHEMA")
df.printSchema()


print("First 50 rows")
df.show(50, truncate=False)


print("Summary Stats")
df.describe().show()


25/11/28 21:57:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


SEC I group SCHEMA
root
 |-- year: string (nullable = true)
 |-- university: string (nullable = true)
 |-- major: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- language: string (nullable = true)
 |-- study_form_symbol: string (nullable = true)
 |-- main_min_score: double (nullable = true)
 |-- main_competition_score: double (nullable = true)
 |-- has_subbakalavr: string (nullable = true)
 |-- sub_min_score: double (nullable = true)
 |-- sub_competition_score: double (nullable = true)

First 50 rows
+---------+---------------------------------------------+--------------------------------------------------+-------------+-----------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|year     |university                                   |major                                             |branch       |language   |study_form_symbol|main_min_score|main_competition_score|has_subbakalavr|sub_min_score|sub_competi

25/11/28 21:57:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+------------------+--------------------+-----------------+-----------+-----------------+------------------+----------------------+---------------+------------------+---------------------+
|summary|     year|        university|               major|           branch|   language|study_form_symbol|    main_min_score|main_competition_score|has_subbakalavr|     sub_min_score|sub_competition_score|
+-------+---------+------------------+--------------------+-----------------+-----------+-----------------+------------------+----------------------+---------------+------------------+---------------------+
|  count|     2180|              2180|                2180|              132|       2180|             2171|              2180|                  1720|           2180|               703|                  325|
|   mean|     NULL|              NULL|                NULL|             NULL|       NULL|             NULL|279.27550458715575|    321.16133720930196|           NULL|  69.94

In [10]:
df.orderBy(df.main_min_score.desc()).show(10, truncate=False)


+---------+---------------------+--------------------------+------+--------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|year     |university           |major                     |branch|language|study_form_symbol|main_min_score|main_competition_score|has_subbakalavr|sub_min_score|sub_competition_score|
+---------+---------------------+--------------------------+------+--------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|2025-2026|ADA Universiteti     |Kompüter elmləri          |NULL  |English |Ə                |692.6         |692.6                 |NO             |NULL         |NULL                 |
|2022-2023|Bakı Ali Neft Məktəbi|İnformasiya təhlükəsizliyi|NULL  |English |Ə                |684.8         |684.8                 |NO             |NULL         |NULL                 |
|2021-2022|Bakı Ali Neft Məktəbi|İnformasiya təhlükəsizliyi|NULL  |English 

In [11]:
df.groupBy("major") \
  .avg("main_min_score", "main_competition_score") \
  .orderBy("avg(main_min_score)", ascending=False) \
  .show(truncate=False)


+--------------------------------------------+-------------------+---------------------------+
|major                                       |avg(main_min_score)|avg(main_competition_score)|
+--------------------------------------------+-------------------+---------------------------+
|Proseslərin avtomatlaşdırılması             |643.0              |643.0                      |
|İnformasiya təhlükəsizliyi                  |515.4              |515.4                      |
|Data analitikası                            |513.4              |531.8749999999999          |
|Kimya mühəndisliyi                          |472.8              |472.8                      |
|Riyaziyyat müəllimliyi                      |472.25             |472.25                     |
|Kompüter elmləri                            |446.3486486486486  |464.5687499999999          |
|Uçuş mühəndisliyi                           |445.2              |445.2                      |
|Uçuş mühəndisliyi                           |443.

In [12]:
df.groupBy("language") \
  .avg("main_min_score") \
  .orderBy("avg(main_min_score)", ascending=False) \
  .show()


+-----------+-------------------+
|   language|avg(main_min_score)|
+-----------+-------------------+
|    English|  402.4477611940296|
|Azerbaijani| 251.50868094701207|
|    Turkish| 215.04999999999998|
+-----------+-------------------+



In [13]:
df.groupBy("university") \
  .avg("main_min_score") \
  .orderBy("avg(main_min_score)", ascending=False) \
  .show(truncate=False)


+------------------------------------------------------------------------+-------------------+
|university                                                              |avg(main_min_score)|
+------------------------------------------------------------------------+-------------------+
|Bakı Ali Neft Məktəbi                                                   |600.8696428571428  |
|Azərbaycan Respublikası Dövlət Gömrük Komitəsinin Akademiyası           |549.4333333333333  |
|Azərbaycan Respublikası Prezidenti yanında Dövlət İdarəçilik Akademiyası|526.925            |
|ADA Universiteti                                                        |526.8428571428572  |
|Türkiyə-Azərbaycan Universiteti                                         |510.0333333333333  |
|Dövlət Gömrük Komitəsinin Akademiyası                                   |493.1              |
|”ADA” Universiteti                                                      |464.29999999999995 |
|“ADA” Universiteti                               

In [14]:
df.filter(df.has_subbakalavr == "YES").show(truncate=False)


+---------+---------------------------------------------+---------------------------------------------------+-------------+-----------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|year     |university                                   |major                                              |branch       |language   |study_form_symbol|main_min_score|main_competition_score|has_subbakalavr|sub_min_score|sub_competition_score|
+---------+---------------------------------------------+---------------------------------------------------+-------------+-----------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|2021-2022|Bakı Dövlət Universiteti                     |Kompüter elmləri                                   |NULL         |Azerbaijani|Q                |250.0         |468.8                 |YES            |77.2         |NULL                 |
|2021-2022|Bakı Dövlət U

In [15]:
df.groupBy("major") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(truncate=False)


+--------------------------------------+-----+
|major                                 |count|
+--------------------------------------+-----+
|İnformasiya texnologiyaları           |164  |
|Kompüter mühəndisliyi                 |145  |
|Ekologiya mühəndisliyi                |92   |
|İnformasiya təhlükəsizliyi            |84   |
|Elektrik və elektronika mühəndisliyi  |83   |
|Kompüter elmləri                      |74   |
|Qida mühəndisliyi                     |66   |
|Energetika mühəndisliyi               |61   |
|Riyaziyyat müəllimliyi                |60   |
|Fizika müəllimliyi                    |59   |
|Riyaziyyat və informatika müəllimliyi |57   |
|Kimya mühəndisliyi                    |56   |
|Mexanika mühəndisliyi                 |47   |
|Sənaye mühəndisliyi                   |47   |
|Proseslərin avtomatlaşdırılması       |43   |
|Torpaqşünaslıq və aqrokimya           |43   |
|İnşaat mühəndisliyi                   |43   |
|Neft-qaz mühəndisliyi                 |39   |
|Logistika və

In [18]:
df.filter(df.university == "Bakı Dövlət Universiteti") \
  .orderBy(df.main_min_score.desc()) \
  .show(30, truncate=False)


+---------+------------------------+--------------------------+------+-----------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|year     |university              |major                     |branch|language   |study_form_symbol|main_min_score|main_competition_score|has_subbakalavr|sub_min_score|sub_competition_score|
+---------+------------------------+--------------------------+------+-----------+-----------------+--------------+----------------------+---------------+-------------+---------------------+
|2022-2023|Bakı Dövlət Universiteti|Riyaziyyat müəllimliyi    |NULL  |English    |Ə                |619.1         |619.1                 |NO             |NULL         |NULL                 |
|2021-2022|Bakı Dövlət Universiteti|Riyaziyyat müəllimliyi    |NULL  |English    |Ə                |605.5         |NULL                  |NO             |NULL         |NULL                 |
|2024-2025|Bakı Dövlət Universiteti|Riyaziyya